In [1]:
import pandas as pd

In [2]:
golden = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/golden_df_trial_bigQuery.csv')

In [18]:
golden['Criminal damage and arson_500'].describe()

count    56.000000
mean      1.142857
std       1.783037
min       0.000000
25%       0.000000
50%       0.000000
75%       2.000000
max      10.000000
Name: Criminal damage and arson_500, dtype: float64

In [8]:
golden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Columns: 125 entries, Unnamed: 0 to latitude
dtypes: float64(21), int64(96), object(8)
memory usage: 54.8+ KB


In [14]:
golden[list(golden.select_dtypes('int64').columns)] = golden[list(golden.select_dtypes('int64').columns)].astype('float64')

In [15]:
golden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Columns: 125 entries, Unnamed: 0 to latitude
dtypes: float64(117), object(8)
memory usage: 54.8+ KB


In [22]:
col_nam = []
for i in list(golden.columns):
    col_nam.append(i.replace(' ', '_'))
golden.columns = col_nam


,Unnamed:_0,District_x,Description_x,GridName_x,lng,lat,hospital_500,train_station_500,bus_station_500,park_500,...,Outdoors_Sub_domain_Score,Outdoors_Sub_domain_Rank__where_1_is_most_deprived_,Outdoors_Sub_domain_Decile__where_1_is_most_deprived_10__of_LSOAs_,Total_population__mid_2015__excluding_prisoners_,Dependent_Children_aged_0_15__mid_2015__excluding_prisoners_,Population_aged_16_59__mid_2015__excluding_prisoners_,Older_population_aged_60_and_over__mid_2015__excluding_prisoners_,Working_age_population_18_59_64__for_use_with_Employment_Deprivation_Domain__excluding_prisoners__,longitude,latitude
0,0.0,Wandsworth London Boro,London Borough,Grid5_0,-0.179675,51.481391,0.0,0.0,0.0,0.0,...,1.795,795.0,1.0,1921.0,245.0,1177.0,499.0,1182.0,-0.172376,51.477924
1,1.0,Wandsworth London Boro,London Borough,Grid8_0,-0.136333,51.481391,0.0,0.0,0.0,0.0,...,1.132,3313.0,2.0,2449.0,571.0,1666.0,212.0,1652.0,-0.139801,51.476509
2,2.0,Wandsworth London Boro,London Borough,Grid1_1,-0.237468,51.472393,0.0,0.0,0.0,0.0,...,1.481,1691.0,1.0,1799.0,348.0,1104.0,347.0,1115.0,-0.225462,51.469482
3,3.0,Wandsworth London Boro,London Borough,Grid6_0,-0.165227,51.481391,0.0,0.0,0.0,0.0,...,1.339,2264.0,1.0,2195.0,354.0,1489.0,352.0,1509.0,-0.166820,51.478434
4,4.0,Wandsworth London Boro,London Borough,Grid7_0,-0.150780,51.481391,0.0,0.0,0.0,0.0,...,1.810,754.0,1.0,1408.0,137.0,1183.0,88.0,1158.0,-0.148750,51.482431
5,5.0,Wandsworth London Boro,London Borough,Grid2_1,-0.223024,51.472393,0.0,0.0,0.0,0.0,...,1.481,1691.0,1.0,1799.0,348.0,1104.0,347.0,1115.0,-0.225462,51.469482
6,6.0,Wandsworth London Boro,London Borough,Grid9_0,-0.121885,51.481391,0.0,0.0,0.0,0.0,...,1.622,1248.0,1.0,1872.0,312.0,1235.0,325.0,1248.0,-0.135980,51.474420
7,7.0,Wandsworth London Boro,London Borough,Grid3_1,-0.208579,51.472393,0.0,0.0,0.0,0.0,...,1.732,933.0,1.0,1661.0,209.0,1181.0,271.0,1203.0,-0.217244,51.466330
8,8.0,Wandsworth London Boro,London Borough,Grid5_1,-0.179690,51.472393,0.0,1.0,0.0,4.0,...,1.410,1941.0,1.0,1323.0,171.0,1039.0,113.0,1049.0,-0.177919,51.470346
9,9.0,Wandsworth London Boro,London Borough,Grid7_1,-0.150801,51.472393,0.0,0.0,0.0,0.0,...,1.858,637.0,1.0,2250.0,480.0,1577.0,193.0,1563.0,-0.152255,51.473787
